In [44]:
import pandas as pd
import sqlite3
import esco_utils as eu
import pandas.io.sql as psql

con = sqlite3.connect('data/cache.db')
esco_con = sqlite3.connect('data/esco_v1.1.0.db')
base_data_path = './model/model_data/'

In [2]:
tasks = pd.read_sql("SELECT * FROM task_data",con)
projects = pd.read_sql("SELECT * FROM projects",con)
tags = pd.read_sql("SELECT * FROM tags",con)
occ2skill = pd.read_sql("SELECT * FROM occupationSkillRelations", con)
skill2skill = pd.read_sql("SELECT * FROM skillSkillRelations", con)
adjSkill = pd.read_sql("SELECT * FROM transversalSkillsCollection", con)

In [3]:
# lang_occupation.head(1)

In [4]:
# p_tasks.merge(tags,right_on='task_id',left_on='_id',how='inner')

In [5]:
all_tags =0

for lindex, lang in enumerate(projects['lang'].unique().tolist()):
    print(lang)
    lang_ocupation = pd.read_sql(f"SELECT * FROM occupations WHERE lang='{lang}'",con)
    lang_ocupation['occupation_id'] = lang_ocupation['occupation_id'].astype(str)
    lang_projs = projects.loc[projects['lang']==lang].loc[projects['total_tasks_labeled']>0].loc[projects['project_name'].str.len()<6]
    l_projs = lang_projs['project_name'].tolist()
    lang_ocupation = lang_ocupation[['iscoGroup','conceptUri', 'preferredLabel','altLabels','description','occupation_id']].rename(columns={"description": "isco_description",'preferredLabel':'isco_preferredLabel','isco_altLabels':'altLabels'})

    for idx,project in enumerate(l_projs):
        p_tasks = tasks.loc[tasks['project_name']==project]
        p_tags = p_tasks.merge(tags,right_on='task_id',left_on='_id',how='inner')
        p_tags = p_tags[['project_name','description','title','occupation_title','occupation_id','task_id']]
        p_tags = p_tags.merge(lang_ocupation,on='occupation_id',how='left')
        p_tags['lang'] = lang
        if idx ==0 and lindex==0:
            lang_tags = p_tags
        else:
            lang_tags = lang_tags.append(p_tags, ignore_index=True)
lang_tags.to_csv(f'./model_data/all_labels.csv')
all_tags+=len(lang_tags)


en
de
fr
nl
el
pt
es
ar


In [18]:

for lindex, lang in enumerate(projects['lang'].unique().tolist()):
    lang_occupation = pd.read_sql(f"SELECT * FROM occupations WHERE lang='{lang}'",con)
    lang_group_occupation = pd.read_sql(f"SELECT * FROM ISCOGroups WHERE lang='{lang}'",con)
    lang_occupation = lang_occupation[['conceptType','conceptUri','preferredLabel','altLabels','description']]
    lang_occupation['lang'] = lang
    lang_group_occupation = lang_group_occupation[['conceptType','conceptUri','preferredLabel','altLabels','description']]
    lang_group_occupation['lang'] = lang
    occupations = lang_group_occupation.append(lang_occupation,ignore_index=True)
    if lindex==0:
        langs_occupations = occupations
    else:
        langs_occupations = langs_occupations.append(occupations, ignore_index=True)
langs_occupations.to_csv(f'./model_data/all_occupations.csv')


In [17]:
for lindex, lang in enumerate(projects['lang'].unique().tolist()):
    lang_occupation = pd.read_sql(f"SELECT * FROM occupations WHERE lang='{lang}'",con)
    lang_group_occupation = pd.read_sql(f"SELECT * FROM ISCOGroups WHERE lang='{lang}'",con)
    lang_occupation = lang_occupation[['conceptType','conceptUri','preferredLabel','altLabels','description']]
    lang_occupation['lang'] = lang
    lang_group_occupation = lang_group_occupation[['conceptType','conceptUri','preferredLabel','altLabels','description']]
    lang_group_occupation['lang'] = lang
    occupations = lang_group_occupation.append(lang_occupation,ignore_index=True)
    if lindex==0:
        langs_occupations = occupations
    else:
        langs_occupations = langs_occupations.append(occupations, ignore_index=True)
langs_occupations.to_csv(f'./model_data/all_occupations.csv')


27869

In [16]:

for lindex, lang in enumerate(projects['lang'].unique().tolist()):
    skill = pd.read_sql(f"SELECT * FROM skills WHERE lang='{lang}'",con)
    skillG = pd.read_sql(f"SELECT * FROM skillGroups WHERE lang='{lang}'",con)

    skill = skill[['conceptType','conceptUri','skillType','reuseLevel','altLabels','description']]
    skill['lang']=lang
    skillG = skillG[['conceptType','conceptUri','preferredLabel','description']]
    skillG['lang']=lang
    skills = skill.append(skillG,ignore_index=True)
    if lindex==0:
        all_skills = skills
    else:
        all_skills = all_skills.append(skills,ignore_index=True)
all_skills.to_csv(f'./model/model_data/all_skills.csv')



In [11]:
'edgesType','nodesTypes','relationType'

,conceptType,conceptUri,preferredLabel,description,lang
0,SkillGroup,http://data.europa.eu/esco/isced-f/00,generic programmes and qualifications,Generic programmes and qualifications are thos...,en


In [20]:
oc_skill = pd.read_sql("SELECT * FROM occupationSkillRelations", esco_con)
oc_skill['nodesTypes']='nonhierarchical-occupation-skill'

In [25]:
non_skill_skill = pd.read_sql("SELECT * FROM occupationSkillRelations", esco_con)
non_skill_skill['nodesTypes']='nonhierarchical-skill-skill'
non_skill_skill.head(1)

,occupationUri,relationType,skillType,skillUri,nodesTypes
0,http://data.europa.eu/esco/occupation/00030d09...,essential,knowledge,http://data.europa.eu/esco/skill/fed5b267-73fa...,nonhierarchical-skill-skill


In [29]:
occ_occ = pd.read_sql("SELECT * FROM broaderRelationsOccPillar",esco_con)
occ_occ['nodesTypes']='hierarchical-occupation-occupation'
occ_occ.head(1)

,conceptType,conceptUri,broaderType,broaderUri,esco_version,nodesTypes
0,ISCOGroup,http://data.europa.eu/esco/isco/C01,ISCOGroup,http://data.europa.eu/esco/isco/C0,v1.0.3,hierarchical-occupation-occupation


In [30]:
skill_skill = pd.read_sql("SELECT * FROM broaderRelationsSkillPillar",esco_con)
skill_skill['nodesTypes']='hierarchical-skill-skill'
skill_skill.head(1)

,conceptType,conceptUri,broaderType,broaderUri,esco_version,nodesTypes
0,SkillGroup,http://data.europa.eu/esco/isced-f/00,SkillGroup,http://data.europa.eu/esco/skill/K,v1.0.3,hierarchical-skill-skill


In [33]:
graph = pd.concat([oc_skill,non_skill_skill,occ_occ,skill_skill])
graph.to_csv('./model_data/graphs.csv')

In [40]:
jobs_con, _ = eu.load_skillLab_DB()

Azuna = psql.read_sql(f"""
        SELECT title, description,location_country 
        FROM jobs 
        WHERE location_country='GB' 
        LIMIT 100000
        """, jobs_con)

In [45]:
Azuna.to_csv(base_data_path+'azuna.csv')